In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

college = pd.read_csv("../../data/College.csv")
college = college.rename({"Unnamed: 0": "College"}, axis=1)
college = college.set_index("College")

college_train, college_valid = train_test_split(college,
                                         test_size=0.2,
                                         random_state=0)


In [3]:
college.Apps.max()

np.int64(48094)

In [4]:
import statsmodels.api as sm
from ISLP.models import ModelSpec as MS 
import numpy as np

X_train = MS(college.columns.drop(["Apps", "Private"])).fit_transform(college_train)
Y_train = college_train["Apps"]

X_valid = MS(college.columns.drop(["Apps", "Private"])).fit_transform(college_valid)
Y_valid = college_valid["Apps"]

model = sm.OLS(Y_train, X_train)
results = model.fit()

Y_hat = results.predict(X_valid)

np.mean((Y_valid - Y_hat)**2)

np.float64(1182113.6667500068)

The validation mean squared error (MSE) of approximately 1,182,114 in the OLS model initially appears large, but when contextualized against the response variable **Apps**, it is more interpretable. Given that the maximum number of applications is around 48,000, this corresponds to a root mean squared error (RMSE) of roughly 1,087 applications, which is about 2.3% of the maximum and likely around 10–15% of the mean. While this level of error may be acceptable for exploratory modeling or rough predictions, it suggests room for improvement, especially if the goal is accurate forecasting.

In [5]:
import sklearn.linear_model as skl
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import sklearn.model_selection as skm

lambdas = 10**np.linspace(8, -2, 100) / Y_train.std()

scaler = StandardScaler(with_mean=True,  with_std=True)

K = 5
kfold = skm.KFold(K,
                  random_state=0,
                  shuffle=True)


lasso = skl.ElasticNetCV(alphas=lambdas, 
                           l1_ratio=1,
                           cv=kfold)

pipe = Pipeline(steps=[('scaler', scaler),
                         ('lasso', lasso)])
pipe.fit(X_train, Y_train)

Y_hat = pipe.predict(X_valid)

np.mean((Y_valid - Y_hat)**2)

np.float64(1182113.6536551272)

In [6]:
tuned_lasso = pipe.named_steps['lasso']
tuned_lasso.coef_

array([ 0.00000000e+00,  4.20865668e+03, -1.04060381e+03,  1.03025296e+03,
       -3.73436230e+02,  4.54799103e+02,  1.05347953e+02, -4.09341509e+02,
        1.63795085e+02,  1.32378568e+00, -8.65477980e+00, -1.39124431e+02,
       -1.55157539e+01,  5.86738679e+01, -2.36632326e+01,  2.88135210e+02,
        1.35564538e+02])

Using the lasso, all of the coefficients were selected, likely due to multicollinearity in the data.

I used RidgeCV instead of ElasticNetCV with l1_ratio=0 because RidgeCV is optimized specifically for that. Using RidgeCV ensures more stable and efficient fitting for pure Ridge models compared to forcing ElasticNetCV to behave like Ridge by setting l1_ratio=0.

In [7]:
from sklearn.linear_model import RidgeCV

ridge = RidgeCV(alphas=lambdas, cv=kfold)
pipe = Pipeline(steps=[('scaler', scaler), ('ridge', ridge)])
pipe.fit(X_train, Y_train)

Y_hat = pipe.predict(X_valid)
mse = np.mean((Y_valid - Y_hat)**2)
print(mse)

1171481.180113388


In [8]:
tuned_ridge = pipe.named_steps['ridge']
tuned_ridge.coef_

array([ 0.00000000e+00,  4.15452579e+03, -9.59256224e+02,  1.00829165e+03,
       -3.56382468e+02,  4.23510221e+02,  1.05074449e+02, -4.01177300e+02,
        1.66750474e+02,  1.68967701e+00, -9.25586209e+00, -1.36324194e+02,
       -1.74295418e+01,  5.96773362e+01, -2.84029748e+01,  2.90316087e+02,
        1.36317895e+02])

The ridge regression model achieved a slightly lower test MSE (1,171,481) compared to both the lasso (1,182,114) and ordinary least squares regression (1,182,114), indicating a modest improvement in predictive performance. This suggests that while the linear model may suffer from multicollinearity or overfitting, the L2 regularization in ridge regression helped stabilize the coefficients without setting any to zero, leading to a better generalization on the validation set. However, the improvement is small, implying that regularization had limited effect in this case, possibly due to a weak signal or all predictors contributing similarly.

In [11]:
from sklearn.decomposition import PCA

pca = PCA()
linreg = skl.LinearRegression()
scaler = StandardScaler(with_mean=True, with_std=True)

pipe = Pipeline([('scaler', scaler),
                 ('pca', pca),
                 ('linreg', linreg)])

max_components = min(X_train.shape[0] * (kfold.get_n_splits() - 1) // kfold.get_n_splits(), X_train.shape[1])

param_grid = {'pca__n_components': range(1, max_components + 1)}

grid = skm.GridSearchCV(pipe,
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')

grid.fit(X_train, Y_train)

best_M = grid.best_params_['pca__n_components']
print(f"Selected number of principal components (M): {best_M}")

Y_pred = grid.predict(X_valid)

test_mse = np.mean((Y_valid - Y_pred)**2)
print(f"Test MSE: {test_mse}")

Selected number of principal components (M): 16
Test MSE: 1182113.666749987


In [10]:
from sklearn.cross_decomposition import PLSRegression

pls = PLSRegression(n_components=2, 
                    scale=True)

param_grid = {'n_components': range(1, max_components + 1)}

grid = skm.GridSearchCV(pls,
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')
grid.fit(X_train, Y_train)

best_M = grid.best_params_['n_components']
print(f"Selected number of principal components (M): {best_M}")

Y_pred = grid.predict(X_valid)

test_mse = np.mean((Y_valid - Y_pred)**2)
print(f"Test MSE: {test_mse}")

Selected number of principal components (M): 16
Test MSE: 1182113.6667499894


Both PCR (Principal Components Regression) and PLS (Partial Least Squares) selected 16 components and produced virtually identical test MSEs (\~1,182,114), indicating no meaningful performance difference between the two methods in this case. This outcome suggests that the dominant variance in the predictor space (captured by PCA) aligns closely with the directions most relevant for predicting the response, which is also what PLS attempts to optimize. As a result, both models arrived at similar solutions despite their conceptual differences—PCR focuses purely on explaining predictor variance, while PLS incorporates the response during component extraction.

Overall, the modeling results on this dataset suggest that regularization and dimension reduction techniques offer only marginal improvements over standard linear regression. The test MSEs from ridge regression, lasso, PCR, and PLS are all very close to one another, with ridge slightly outperforming the others. This indicates that the predictors are likely moderately multicollinear, but not in a way that severely impacts the performance of ordinary least squares. The fact that lasso did not shrink any coefficients to zero, and both PCR and PLS selected the maximum number of components, further supports the idea that no strong subset of predictors dominates the relationship with the response. Thus, while regularization and dimension reduction offer robustness, the underlying signal in the data is relatively diffuse across predictors, and no method provides a clear advantage.